# Введение в анализ данных (первый семестр)
# Задание №4

*** Конопаткин Олег, BD11 ***

In [1]:
import requests
import json
import html
import urllib
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
from selenium import webdriver
from time import sleep

def get_books_urls(driver, url, t_sleep=1):
    driver.get(url)
    sleep(t_sleep)
    
    while True:
        for i in range(3):
            driver.execute_script("window.scrollTo(0, 10000000);")
            sleep(3)
        try:
            elem = driver.find_element_by_class_name('js__show-more-cards')
            elem.click()
        except:
            break
    
    urls = []
    
    items = driver.find_elements_by_class_name("product-card__img")
    
    for item in items:
        url = item.get_attribute("href")
        urls.append(url[:url.rfind('/')])
    
    return urls

In [3]:
# собираем страницы с книгами

authors = ['Фрай М.', 'Хантер Э.', 'Емец Д.']
card_urls = []
base_url = "https://www.chitai-gorod.ru/search/result.php?q={}&type=author"

with webdriver.Chrome(executable_path='C:\\Anaconda3\\chromedriver.exe') as driver:
    for author in authors:
        url = base_url.format(urllib.parse.quote(author, encoding='windows-1251'))
        cur_urls = get_books_urls(driver, url)
        print(len(cur_urls))
        card_urls.extend(cur_urls)

153
97
192


In [4]:
card_ids = [int(url[url.rfind('/') + 1:]) for url in card_urls]
print(card_ids)

[244296, 245728, 246246, 246247, 246533, 246749, 246913, 246914, 247198, 247199, 247200, 247406, 248903, 253620, 256186, 259037, 259751, 259790, 266507, 266508, 268263, 270861, 271107, 271108, 272859, 274724, 274979, 280462, 291153, 292170, 293276, 296766, 300551, 300799, 301879, 302904, 305522, 305523, 309851, 310895, 310896, 312706, 323381, 324455, 350170, 353710, 354139, 354742, 358088, 358727, 361307, 361874, 364589, 367135, 376401, 377037, 379158, 389640, 389641, 392051, 392052, 400412, 411548, 411549, 411550, 411551, 411552, 411553, 411625, 411626, 411627, 411628, 411629, 411682, 411683, 411692, 411693, 411732, 411733, 411750, 442981, 442982, 471783, 710328, 723716, 765102, 779618, 784979, 789215, 790009, 793750, 810832, 812827, 816107, 818391, 820823, 822874, 824732, 825450, 828740, 831871, 839265, 843835, 847078, 848994, 850216, 855443, 857607, 860114, 861223, 867637, 868652, 870661, 870662, 870663, 872961, 875812, 882380, 886599, 888578, 894293, 898955, 901916, 906001, 915767,

In [5]:
def get_text(soup, tag, class_):
    text = soup.find(tag, class_=class_)
    if text is not None:
        return text.text.strip()
    return text

def extract_book_info(card_id):
    # print('\r{} from {};'.format(card_ids.index(card_id) + 1, len(card_ids)), end = ' ')
    card_url = 'https://www.chitai-gorod.ru/catalog/book/{}'.format(card_id)
    while True:
        card_html = requests.get(card_url).text
        if card_html.find('Request unsuccessful') == -1:
            break
        # print(card_html)
        sleep(5)
    soup = BeautifulSoup(card_html, 'html.parser')
    
    titles = list(map(lambda f: f.text.strip(), soup.find_all('div', class_='product-prop__title')))
    values = list(map(lambda f: f.text.strip(), soup.find_all('div', class_='product-prop__value')))
    result = {t: v for (t, v) in zip(titles, values)}
    
    result['ID карточки'] = card_id
    
    result['Название'] = get_text(soup, 'h1', 'product__title')
    result['Автор'] = get_text(soup, 'a', 'product__author')
    
    rating = get_text(soup, 'div', 'rating')
    result['Рейтинг'] = float(rating[:rating.find('(')])
    result['Голоса'] = int(rating[rating.find('(') + 1:rating.find(')')])
    
    price = soup.find('div', class_='trade-container')
    price = price.find('meta', itemprop='price').attrs['content']
    result['Цена'] = int(price)
    
    image_link = soup.find('div', class_='product__image')
    if image_link is not None:
        image_link = image_link.find('img').attrs['src']
    result['Обложка'] = image_link
    
    # print('\r{} from {}; elapsed time: {}'.format(card_ids.index(card_id) + 1, len(card_ids), 
    #                                               round(time() - start, 3)), end = ' ')
    res[card_id] = result
    return

In [6]:
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool

from time import time
start = time()

#result = list(map(lambda id_: extract_book_info(id_), card_ids))

res = {card_id: {} for card_id in card_ids}

t = ThreadPool(16)
r = t.map(lambda card_id: extract_book_info(card_id), card_ids)
t.close()
t.join()

result = list(res.values())

print('total time: {}'.format(time() - start))

total time: 82.55272197723389


In [7]:
df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

cols = df.columns.tolist()
cols.remove('Обложка')
cols.append('Обложка')
df = df[cols]

with open('data/hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [8]:
df

,ID карточки,ID товара:,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена,Обложка
0,244296,1400705,9785367009781,Фрай М.,NaN,2009,0,"Амфора, ООО",478,"Книга для таких, как я",NaN,NaN,0.0,ФРАМ,Твердая бумажная,4000,20.5 x 13.2 x 3,NaN,322,https://img-gorod.ru/upload/iblock/73b/73b0845...
1,245728,1665067,9785942783372,Фрай М.,NaN,2010,0,"Амфора, ООО",606,Волонтеры вечности,NaN,NaN,0.0,NaN,Твердая бумажная,3000,20.7 x 13.8 x 3.8,NaN,336,https://img-gorod.ru/upload/iblock/bd5/bd5917e...
251,246061,1668112,9785699110018,Емец Д.,NaN,2003,0,Эксмо-Пресс,384,Таня Гроттер и Золотая Пиявка,NaN,NaN,0.0,Таня Гроттер,Твердая бумажная,NaN,20.6 x 13.5 x 2.3,NaN,154,https://img-gorod.ru/upload/iblock/236/236a031...
2,246246,1801350,5942783535,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Простые волшебные вещи,NaN,NaN,0.0,Лабиринты Эхо,Твердая бумажная,NaN,20.8 x 13.7 x 2.3,NaN,223,https://img-gorod.ru/upload/iblock/82d/82d0b11...
3,246247,1801351,9785367016239,Фрай М.,NaN,2009,0,"Амфора, ООО",367,Темная сторона,NaN,NaN,0.0,Лабиринты Ехо,Твердая бумажная,3000,20.6 x 13.5 x 2.3,NaN,336,https://img-gorod.ru/upload/iblock/a10/a1063f0...
4,246533,1804075,9785942783600,Фрай М.,12+,2013,1,"Амфора, ООО",639,Чужак,NaN,NaN,5.0,Лабиринты Ехо,Твердая бумажная,3000,20.7 x 13.6 x 4.1,NaN,336,https://img-gorod.ru/upload/iblock/2ae/2ae746b...
5,246749,1806167,9785942783877,Фрай М.,12+,2013,0,"Амфора, ООО",590,Болтливый мертвец,NaN,Стукалина Марина,0.0,Лабиринты Эхо,Твердая бумажная,2000,20.7 x 13.8 x 3.9,NaN,367,https://img-gorod.ru/upload/iblock/e40/e406f84...
6,246913,1807965,9785367003390,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Власть несбывшегося,NaN,NaN,0.0,NaN,Твердая бумажная,NaN,20.6 x 13.6 x 2.3,NaN,322,https://img-gorod.ru/upload/iblock/052/052c143...
7,246914,1807966,9785942783884,Фрай М.,12+,2009,0,"Амфора, ООО",383,Наваждения,NaN,NaN,0.0,Лабиринты Ехо,Твердая бумажная,5000,20.6 x 13.7 x 2.5,NaN,427,https://img-gorod.ru/upload/iblock/6c2/6c220c8...
8,247198,1810879,9785367004069,Фрай М.,NaN,2003,0,"Амфора, ООО",557,Гнезда Химер Хроники Овётганны,NaN,NaN,0.0,NaN,Твердая бумажная,NaN,20.7 x 13.8 x 3.4,NaN,322,https://img-gorod.ru/upload/iblock/792/7923d42...
